----
# I. Data preprocessing

## 1. Read dataset

In [1]:
import pandas as pd

train = pd.read_csv("../dataset/train.csv")
test = pd.read_csv("../dataset/test.csv")
submission = pd.read_csv("../dataset/sample_submission.csv")

## 2. Split `train` to `sub_train` and `sub_test`

In [2]:
sub_test = pd.DataFrame()

for store in range(1, train["Store"].max() + 1):
    splited = train[train["Store"] == store].tail(4)
    train = train.drop(splited.index.to_list())
    sub_test = pd.concat([sub_test, splited])

sub_test = sub_test.reset_index(drop=1)
sub_train = train.reset_index(drop=1)

In [3]:
sub_train

,id,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales
0,1,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,8.106,False,1643690.90
1,2,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,8.106,True,1641957.44
2,3,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,8.106,False,1611968.17
3,4,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,8.106,False,1409727.59
4,5,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,8.106,False,1554806.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6070,6247,45,03/08/2012,76.58,3.654,24853.05,39.56,17.96,11142.69,2768.32,8.684,False,725729.51
6071,6248,45,10/08/2012,78.65,3.722,17868.84,50.60,57.66,2593.93,1890.59,8.684,False,733037.32
6072,6249,45,17/08/2012,75.71,3.807,3657.79,6.00,0.30,1630.50,3794.22,8.684,False,722496.93
6073,6250,45,24/08/2012,72.62,3.834,7936.20,58.38,22.00,5518.07,2291.97,8.684,False,718232.26


In [4]:
sub_test

,id,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales
0,136,1,07/09/2012,83.96,3.730,5204.68,35.74,50.94,4120.32,2737.17,6.908,True,1661767.33
1,137,1,14/09/2012,74.97,3.717,17212.52,7.00,18.79,1523.11,7992.72,6.908,False,1517428.87
2,138,1,21/09/2012,69.87,3.721,6352.30,7.64,4.69,1010.06,6456.71,6.908,False,1506126.06
3,139,1,28/09/2012,76.08,3.666,3666.27,7.64,1.65,1417.96,4744.28,6.908,False,1437059.26
4,275,2,07/09/2012,87.65,3.730,10658.86,149.28,60.97,2920.90,3275.57,6.565,True,1898777.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,6116,44,28/09/2012,64.80,3.821,380.96,NaN,NaN,NaN,2022.03,5.407,False,355307.94
176,6252,45,07/09/2012,75.70,3.911,11024.45,12.80,52.63,1854.77,2055.70,8.684,True,766512.66
177,6253,45,14/09/2012,67.87,3.948,11407.95,NaN,4.30,3421.72,5268.92,8.684,False,702238.27
178,6254,45,21/09/2012,65.32,4.038,8452.20,92.28,63.24,2376.38,8670.40,8.684,False,723086.20


## 2. Missing values handling

### a) `sub_train`

In [5]:
sub_train = sub_train.fillna(0)

### b) `sub_test` and `test`

In [6]:
def each_means(df):
    means = {}
    for i in range(1, 6):
        means[f"Promotion{i}"] = df[f"Promotion{i}"].mean()
    return means

sub_test = sub_test.fillna(value=each_means(sub_test))
test = test.fillna(value=each_means(test))

## 3. Data manipulation

### a) `Date`

In [7]:
import datetime as dt

def date_to_week(date):
    day, month, year = map(int, date.split('/'))
    t = dt.datetime(year, month, day) - dt.datetime(2010, 2, 5)
    return t.days // 7


def date_split(date):
    return tuple(map(int, date.split('/')))


sub_train["Day"] = sub_train["Date"].apply(lambda x: date_split(x)[0])
sub_train["Month"] = sub_train["Date"].apply(lambda x: date_split(x)[1])
sub_train["Year"] = sub_train["Date"].apply(lambda x: date_split(x)[2])

sub_test["Day"] = sub_test["Date"].apply(lambda x: date_split(x)[0])
sub_test["Month"] = sub_test["Date"].apply(lambda x: date_split(x)[1])
sub_test["Year"] = sub_test["Date"].apply(lambda x: date_split(x)[2])

test["Day"] = test["Date"].apply(lambda x: date_split(x)[0])
test["Month"] = test["Date"].apply(lambda x: date_split(x)[1])
test["Year"] = test["Date"].apply(lambda x: date_split(x)[2])

### b) `IsHoliday`

In [8]:
sub_train["IsHoliday"] = sub_train["IsHoliday"].apply(int)
sub_test["IsHoliday"] = sub_test["IsHoliday"].apply(int)
test["IsHoliday"] = test["IsHoliday"].apply(int)

### c) `Store`

In [9]:
sub_train = pd.get_dummies(data=sub_train, columns=["Store"])
sub_test = pd.get_dummies(data=sub_test, columns=["Store"])
test = pd.get_dummies(data=test, columns=["Store"])

### d) `Promotion1`, ... , `Promotion5`

In [10]:
# Scaling
from sklearn.preprocessing import RobustScaler


promos = ['Promotion1','Promotion2','Promotion3','Promotion4','Promotion5']

scaler = RobustScaler()
scaler.fit(sub_train[promos])
sub_train[promos] = scaler.transform(sub_train[promos])
sub_test[promos] = scaler.transform(sub_test[promos])
test[promos] = scaler.transform(test[promos])

### e) `Weekly_Sales` 

In [11]:
import numpy as np

sub_train["Weekly_Sales"] = np.log1p(sub_train["Weekly_Sales"])

## 4. Remove not using features

In [12]:
remove = ['id','Date']

sub_train = sub_train.drop(columns=remove)
sub_test = sub_test.drop(columns=remove)
test = test.drop(columns=remove)

In [13]:
sub_train

,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales,...,Store_36,Store_37,Store_38,Store_39,Store_40,Store_41,Store_42,Store_43,Store_44,Store_45
0,42.31,2.572,0.000000,0.00,0.000000,0.000000,0.000000,8.106,0,14.312455,...,0,0,0,0,0,0,0,0,0,0
1,38.51,2.548,0.000000,0.00,0.000000,0.000000,0.000000,8.106,1,14.311400,...,0,0,0,0,0,0,0,0,0,0
2,39.93,2.514,0.000000,0.00,0.000000,0.000000,0.000000,8.106,0,14.292967,...,0,0,0,0,0,0,0,0,0,0
3,46.63,2.561,0.000000,0.00,0.000000,0.000000,0.000000,8.106,0,14.158908,...,0,0,0,0,0,0,0,0,0,0
4,46.50,2.625,0.000000,0.00,0.000000,0.000000,0.000000,8.106,0,14.256862,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6070,76.58,3.654,23.066225,39.56,9.047859,210.577152,1.786779,8.684,0,13.494934,...,0,0,0,0,0,0,0,0,0,1
6071,78.65,3.722,16.584149,50.60,29.047859,49.020694,1.220259,8.684,0,13.504953,...,0,0,0,0,0,0,0,0,0,1
6072,75.71,3.807,3.394811,6.00,0.151134,30.813569,2.448935,8.684,0,13.490470,...,0,0,0,0,0,0,0,0,0,1
6073,72.62,3.834,7.365622,58.38,11.083123,104.281773,1.479325,8.684,0,13.484550,...,0,0,0,0,0,0,0,0,0,1


In [14]:
sub_test

,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales,...,Store_36,Store_37,Store_38,Store_39,Store_40,Store_41,Store_42,Store_43,Store_44,Store_45
0,83.96,3.730,4.830486,35.740000,25.662469,77.866767,1.766674,6.908,1,1661767.33,...,0,0,0,0,0,0,0,0,0,0
1,74.97,3.717,15.975015,7.000000,9.465995,28.784088,5.158807,6.908,0,1517428.87,...,0,0,0,0,0,0,0,0,0,0
2,69.87,3.721,5.895598,7.640000,2.362720,19.088349,4.167407,6.908,0,1506126.06,...,0,0,0,0,0,0,0,0,0,0
3,76.08,3.666,3.402681,7.640000,0.831234,26.796938,3.062140,6.908,0,1437059.26,...,0,0,0,0,0,0,0,0,0,0
4,87.65,3.730,9.892535,149.280000,30.715365,55.199849,2.114178,6.565,1,1898777.07,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,64.80,3.821,0.353571,39.962783,21.414080,32.316941,1.305095,5.407,0,355307.94,...,0,0,0,0,0,0,0,0,1,0
176,75.70,3.911,10.231840,12.800000,26.513854,35.051876,1.326827,8.684,1,766512.66,...,0,0,0,0,0,0,0,0,0,1
177,67.87,3.948,10.587769,39.962783,2.166247,64.664462,3.400762,8.684,0,702238.27,...,0,0,0,0,0,0,0,0,0,1
178,65.32,4.038,7.844524,92.280000,31.858942,44.909383,5.596207,8.684,0,723086.20,...,0,0,0,0,0,0,0,0,0,1


In [15]:
test

,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Day,...,Store_36,Store_37,Store_38,Store_39,Store_40,Store_41,Store_42,Store_43,Store_44,Store_45
0,68.55,3.617,7.497125,64.672,9.178841,68.363035,2.340449,6.573,0,5,...,0,0,0,0,0,0,0,0,0,0
1,62.99,3.601,1.936193,64.672,4.085642,11.383540,3.825157,6.573,0,12,...,0,0,0,0,0,0,0,0,0,0
2,67.97,3.594,0.882005,64.672,2.483627,1.516583,1.492802,6.573,0,19,...,0,0,0,0,0,0,0,0,0,0
3,69.16,3.506,2.399939,31.750,3.022670,19.978456,0.842303,6.573,0,26,...,0,0,0,0,0,0,0,0,0,0
4,70.27,3.617,5.603672,64.672,5.057935,57.211944,2.487132,6.170,0,5,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,46.97,3.755,0.035871,2.610,0.493703,24.271758,0.295443,5.217,0,26,...,0,0,0,0,0,0,0,0,1,0
176,64.89,3.985,4.683902,64.672,9.481108,42.585845,1.510332,8.667,0,5,...,0,0,0,0,0,0,0,0,0,1
177,54.47,4.000,1.815632,64.672,3.974811,11.326089,2.575647,8.667,0,12,...,0,0,0,0,0,0,0,0,0,1
178,56.47,3.969,1.859940,64.672,1.602015,8.272324,0.992355,8.667,0,19,...,0,0,0,0,0,0,0,0,0,1


----
# II. Modeling

## 1. Divide `sub_train` and `sub_test`

In [16]:
x_train = sub_train.drop(columns=["Weekly_Sales"])
y_train = sub_train["Weekly_Sales"]

x_eval = sub_test.drop(columns=["Weekly_Sales"])
y_eval = sub_test["Weekly_Sales"]

## 2. Choose a suitable model

In [17]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error


# Below submission get `49,425.30556` score.
model = xgb.XGBRegressor(
    objective='reg:squarederror', 
    learning_rate=0.1, 
    max_depth = 4, 
    n_estimators = 1000
)

model.fit(x_train, y_train)
pred = np.expm1(model.predict(x_eval))
rmse = mean_squared_error(y_eval.to_list(), pred) ** 0.5
rmse

53217.74427803348

### a) Hyper-parameter tuning

In [18]:
!pip install bayesian-optimization

from bayes_opt import BayesianOptimization


def xgb_cv(max_depth, learning_rate, n_estimators, gamma, min_child_weight, subsample,\
                                                       colsample_bytree, silent=True, nthread=-1):
    
    model = xgb.XGBRegressor(
        objective='reg:squarederror',
        max_depth = int(max_depth),
        learning_rate = learning_rate,
        n_estimators = int(n_estimators),
        gamma = gamma,
        min_child_weight = min_child_weight,
        subsample = subsample,
        colsample_bytree = colsample_bytree, 
        nthread = nthread)
    
    model.fit(x_train, y_train)
    y_pred = np.expm1(model.predict(x_eval))
    rmse = mean_squared_error(y_eval.to_list(), y_pred) ** 0.5
    print(f"RMSE => {rmse}")
    return -rmse


pbounds = {
    'max_depth': (2, 8), 
    'learning_rate': (0.01, 0.5),
    'n_estimators': (100, 1000),
    'gamma': (0, 10),
    'min_child_weight': (0, 5),
    'subsample': (0.5, 1),
    'colsample_bytree' :(0.2, 1)
}

#optimizer = BayesianOptimization(f=xgb_cv, pbounds=pbounds, verbose=1, random_state=1)
#optimizer.maximize(init_points=2, n_iter=1000, acq='ei', xi=0.01)
#print(optimizer.max)

In [19]:
from sklearn.model_selection import GridSearchCV

param_xgb = {"max_depth": [3,4,5,6],
              "min_child_weight" : [1,3,6,10],
              "n_estimators": [100,200,300,500]
            }

model = xgb.XGBRegressor()
gscv_xgb = GridSearchCV(estimator = model, param_grid = param_xgb, 
                        scoring ="neg_root_mean_squared_error", 
                        cv = 3, refit=True, n_jobs=-1, verbose=3)

gscv_xgb.fit(x_train, y_train)

print('best parameters : ', gscv_xgb.best_params_)
print('best score : ', gscv_xgb.best_score_)
model = gscv_xgb.best_estimator_
pred = np.expm1(model.predict(x_eval))
rmse = mean_squared_error(y_eval.to_list(), pred) ** 0.5
rmse

Fitting 3 folds for each of 64 candidates, totalling 192 fits
best parameters :  {'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100}
best score :  -0.593948606712894


82130.72729051055